# **作业一 PaddleX的客户端体验**
![](https://ai-studio-static-online.cdn.bcebos.com/3673db7a5da54cf48babe4f0537e54abcd3725d8b0a145bd9f94e42ed00c9c62)
![](https://ai-studio-static-online.cdn.bcebos.com/1da864afdbfb483e9c33b574fef3a93ec4fc0e216e144576a542e5c079200a31)
![](https://ai-studio-static-online.cdn.bcebos.com/324cf832b8f2440ba853683aae355cae305587b924994220bc996a154706c491)
![](https://ai-studio-static-online.cdn.bcebos.com/bf3e4c063df4410495d60b2fc48f8ad2ebe2c4cc5e784fe9a85ed6d1e079a971)
![](https://ai-studio-static-online.cdn.bcebos.com/3ba8b3d72d9141ba9fae2f2d87f0dbd38eb33ea086ca4c1682d9a9a83144f37e)
![](https://ai-studio-static-online.cdn.bcebos.com/073b36d95ca942f5a59629192f1cb8eec1682a98ca3447b084a317cfb5e63f53)
![](https://ai-studio-static-online.cdn.bcebos.com/5361a7d8c444444faf4c4ea3a4ab24f5e6d13159756c4f33a2501c17d594ea4c)


# **作业二 基于PaddleX API的语义分割**
### 1.安装paddleX

In [ ]:
! pip install paddlex -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 522kB 10.4MB/s eta 0:00:01
     |████████████████████████████████| 153kB 32.9MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 30.3MB/s eta 0:00:01
     |████████████████████████████████| 102kB 27.2MB/s ta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=278363 sha256=fd307cd52d1001f4e472cbecd26b3f3d74244285912bd0bcf4ba60e1b40e8283
  Stored in directory: /home/aistudio/.cache/pip/wheels/fb/44/67/8baa69040569b1edbd7776ec6f82c387663e724908aaa60963
Successfully built pycocotools


### 2.设置数据集
解压缩-->划分数据集

In [2]:
! unzip -oq /home/aistudio/data/data81942/00.zip

In [3]:
!paddlex --data_conversion --source labelme --to MSCOCO \
        --pics 00 \
        --annotations 00 \
        --save_dir dataset_00

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [4]:
# 按照COCO的方式划分数据集
!paddlex --split_dataset --format COCO --dataset_dir dataset_00 --val_value 0.2 --test_value 0.1

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is depre

### 3.开始训练

In [5]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx
from paddlex.det import transforms

# 定义训练和验证时的transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(), transforms.Normalize(),
    transforms.ResizeByShort(
        short_size=800, max_size=1333), transforms.Padding(coarsest_stride=32)
])

eval_transforms = transforms.Compose([
    transforms.Normalize(), transforms.ResizeByShort(
        short_size=800, max_size=1333), transforms.Padding(coarsest_stride=32)
])

# 定义训练和验证所用数据集
train_dataset = pdx.datasets.CocoDetection(
    data_dir='dataset_00/JPEGImages',
    ann_file='dataset_00/train.json',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.CocoDetection(
    data_dir='dataset_00/JPEGImages',
    ann_file='dataset_00/val.json',
    transforms=eval_transforms)

# 初始化模型并训练
num_classes = len(train_dataset.labels) + 1

model = pdx.det.MaskRCNN(num_classes=num_classes, backbone='ResNet50')

model.train(
    num_epochs=100,
    train_dataset=train_dataset,
    train_batch_size=4,
    eval_dataset=eval_dataset,
    learning_rate=0.00125,
    warmup_steps=10,
    lr_decay_epochs=[8, 11],
    save_interval_epochs=10,
    save_dir='output/mask_rcnn_r50_fpn',
    use_vdl=True)


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
2021-04-20 22:35:47 [INFO]	Starting to read file list from dataset...
2021-04-20 22:35:47 [INFO]	293 samples in file dataset_00/train.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
2021-04-20 22:35:47 [INFO]	Starting to read file list from dataset...
2021-04-20 22:35:47 [INFO]	83 samples in file dataset_00/val.json


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/detection/fpn.py:172
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
2021-04-20 22:35:48,908 - INFO - If regularizer of a Parameter has been set by 'fluid.ParamAttr' or 'fluid.WeightNormParamAttr' already. The Regularization[L2Decay, regularization_coeff=0.000100] in Optimizer will not take effect, and it will only be applied to other Parameters!
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-pad

2021-04-20 22:35:49 [INFO]	Downloading ResNet50_cos_pretrained.tar from https://paddle-imagenet-models-name.bj.bcebos.com/ResNet50_cos_pretrained.tar


  0%|          | 0/100310 [00:00<?, ?KB/s]100%|██████████| 100310/100310 [00:01<00:00, 69910.05KB/s]


...

100%|██████████| 83/83 [00:15<00:00,  5.52it/s]


creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.08s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.238
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.648
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.041
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.238
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.273
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.418
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.418
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0

100%|██████████| 83/83 [00:14<00:00,  5.53it/s]


creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.243
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.666
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.055
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.243
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.287
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.411
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0

KeyboardInterrupt: 

### 4.测试模型

In [7]:
model = pdx.load_model('output/mask_rcnn_r50_fpn/best_model')
image_name = '00/lhgd225.jpg'
result = model.predict(image_name)
pdx.det.visualize(image_name, result, threshold=0.5)

2021-04-20 22:57:47 [INFO]	Model[MaskRCNN] loaded.
2021-04-20 22:57:47 [INFO]	The visualized result is saved as ./visualize_lhgd225.jpg


![](https://ai-studio-static-online.cdn.bcebos.com/3a4b8d3d3de744a1a0dbcf2de69955c8ece8bde3ab28467fb5229ddfde4810e2)



